# BLIP 的模型结构
图像编码器（Visual Encoder）：采用 12 层的 Vision Transformer（ViT-Base），将输入图像转换为视觉特征嵌入，捕捉图像全局及局部语义信息。
文本编码器（Text Encoder）：基于 BERT 模型构建，分为多个子模块，分别负责不同任务：
第一个文本编码器：通过双向自注意力（Self-Attention）处理文本，输出含全局信息的 [CLS] 嵌入，与图像编码器的输出联合进行图文对比学习（ITC）。
第二个文本编码器：引入交叉注意力（Cross-Attention）与图像特征交互，输出融合特征用于图文匹配任务（ITM），判断图文对是否匹配（二分类）。
文本解码器（Text Decoder）：基于 BERT 的后半部分改造，采用因果自注意力（Causal Self-Attention）（类似 GPT 的机制），仅允许关注前文信息，用于语言生成任务（LM）。其核心是通过 “续写式” 建模（而非 BERT 的完形填空），支持图像描述生成、问答等生成式任务。
模型通过三个核心任务联合训练，实现理解与生成的统一：
ITC（Image-Text Contrastive Loss）：通过图像与文本的全局嵌入对比，拉近匹配对距离，拉远非匹配对距离，增强跨模态语义对齐。
ITM（Image-Text Matching Loss）：通过二分类判断图文对是否匹配，提升模型对图文关联性的识别能力。
LM（Language Modeling Loss）：基于解码器的因果自注意力，通过 “续写” 任务训练生成能力（如给定图像生成描述、给定问题生成答案）。

# BLIP 的创新点
BLIP 的创新主要体现在模型结构设计和数据去噪机制两方面，解决了传统模型 “无法兼顾生成与理解任务” 及 “大规模数据含噪声” 的核心问题。
1. 多模态混合编码 - 解码结构（MED）：统一生成与理解任务
痛点解决：传统模型中，编码器模型（如 ALBEF）擅长理解任务（检索、匹配）但无法生成文本；编码器 - 解码器模型擅长生成任务但难以高效完成理解任务。
创新设计：BLIP 通过 MED 结构将 “理解” 与 “生成” 整合到同一框架：
文本编码器负责理解任务（ITC、ITM），通过双向自注意力和交叉注意力捕捉图文语义关联；
文本解码器负责生成任务（LM），通过因果自注意力实现 “续写式” 语言建模（类似 GPT），支持图像描述、问答等生成任务。
效果：首次在单一模型中同时支持检索、匹配等理解任务和描述生成、问答等生成任务，提升了多模态模型的通用性。
2. Captioning and Filtering（CapFilt）：大规模噪声数据去噪
痛点解决：从网络爬取的大规模图文对含大量噪声（如图文匹配度低），直接训练会影响模型性能。
创新机制：通过 “生成 + 过滤” 的双阶段策略构建高质量数据集，具体流程如下：
过滤（Filtering）：用预训练的图文编码器对原始网络数据（{I_w, T_w}）进行筛选，保留匹配度高的文本（T_w^f），剔除噪声样本。
生成（Captioning）：用图像描述生成器对网络图像（I_w）生成合成文本（T_s），补充高质量描述。
二次过滤：对合成文本（T_s）再次用编码器筛选，保留与图像匹配度高的高质量合成文本（T_s^f）。
数据自举（Bootstrapping）：将过滤后的网络数据（{I_w, T_w^f}）、高质量合成数据（{I_w, T_s^f}）与人工标注数据（{I_h, T_h}）合并，形成新训练集，迭代优化模型。
效果：在无需大规模人工标注的情况下，显著提升数据质量，同时通过生成扩充数据量，实验表明仅用 14M 数据即可超越 ALBEF 等模型，129M 数据性能比肩使用 1.8B 数据的 SimVLM。
3. 硬负样本与任务协同的强化
在 ITM 任务中，负样本来自图文对比学习中相似度次高的样本（硬负样本），增强模型对细微差异的辨别能力，进一步提升图文匹配精度。